<a href="https://colab.research.google.com/github/Hassan-zeidan/AAI614_Zeidan/blob/main/Week%203/assignment%205/Outlier_Detection_With_IQR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Try the algorithm on a dataset of your choice and share the results as well. Do these outliers make sense in the data context?

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [13]:
s = df['SalePrice']


,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


In [26]:
Q1 = s.quantile(0.25)
Q2 = s.quantile(0.5)
Q3 = s.quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
(lower, upper)

outlier_mask = (s < lower) | (s > upper)
outliers = df[outlier_mask]

len_outliers = outliers.shape[0]
outliers[["SalePrice"]].head()

,SalePrice
11,345000
53,385000
58,438780
112,383970
151,372402


In [28]:
outliers = df[outlier_mask].copy()
len_outliers, lower, upper

(61, np.float64(3937.5), np.float64(340037.5))

In [34]:
outliers[["SalePrice","GrLivArea","OverallQual","YearBuilt","LotArea","SaleCondition"]].head(10)

,SalePrice,GrLivArea,OverallQual,YearBuilt,LotArea,SaleCondition
11,345000,2324,9,2005,11924,Partial
53,385000,1842,9,1981,50271,Normal
58,438780,2945,10,2006,13682,Partial
112,383970,2696,7,2007,9965,Partial
151,372402,1710,8,2007,13891,Partial
161,412500,2668,9,2003,13688,Normal
178,501837,2234,9,2008,17423,Partial
185,475000,3608,10,1892,22950,Normal
224,386250,2392,10,2003,13472,Normal
231,403000,2794,8,1995,15138,Normal


In [36]:
cols = ["SalePrice","GrLivArea","OverallQual","YearBuilt","LotArea"]
summary = pd.DataFrame({
    "all_median": df[cols].median(numeric_only=True),
    "outlier_median": df.loc[outlier_mask, cols].median(numeric_only=True),
    "inlier_median": df.loc[~outlier_mask, cols].median(numeric_only=True)
})
summary

,all_median,outlier_median,inlier_median
SalePrice,163000.0,394617.0,159500.0
GrLivArea,1464.0,2364.0,1437.0
OverallQual,6.0,9.0,6.0
YearBuilt,1973.0,2006.0,1971.0
LotArea,9478.5,13682.0,9317.0


I used the IQR rule on SalePrice to flag outliers. When I compared groups, the outliers had a much higher median price (394,617 vs 159,500 for inliers). They were also bigger homes (GrLivArea 2364 vs 1437), higher quality (OverallQual 9 vs 6), newer (YearBuilt 2006 vs 1971), and on larger lots (13682 vs 9317). In the housing context, this pattern looks like legitimate luxury properties
So these outliers make sense because they’re simply the high-end edge of the market